# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
import zipfile

orders = pd.read_csv('orders.zip',compression='zip')

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [106]:
# your code here
# sub problem 1
# We'll group the values by customerid and aggregate the sum of amount_spent. In this way we'll have the total expenses per unique customer
customer_amount=orders.groupby('CustomerID').agg({'amount_spent':'sum', 'Country':'max'})
customer_amount['amount_spent'].sort_values(ascending=False)

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
12415    124914.53
14156    117379.63
17511     91062.38
16029     81024.84
12346     77183.60
16684     66653.56
14096     65164.79
13694     65039.62
15311     60767.90
13089     58825.83
17949     58510.48
15769     56252.72
15061     54534.14
14298     51527.30
14088     50491.81
15749     44534.30
12931     42055.96
17841     40991.57
15098     39916.50
13798     37153.85
16013     37130.60
16422     34684.40
12748     33719.73
15838     33643.08
17404     31906.82
           ...    
14589        39.75
12977        39.24
16659        39.12
18019        38.45
12855        38.10
16162        37.40
17816        36.56
15940        35.80
17831        35.40
16856        35.40
14576        35.40
15744        34.80
16765        34.00
17408        32.65
13120        30.60
15657        30.00
18268        25.50
17102        25.50
17986        20.80
16953        20.80
16093        17.00
1

In [107]:
# sub problem 2 & 3
# First we'll find the values that correspond to the quantile 0.95 and 0.75
q95=customer_amount.quantile(q=0.95, axis=0)[0]
q75=customer_amount.quantile(q=0.75, axis=0)[0]

# Then apply a function to create a new column with values 'VIP' or 'Preferred' depending on the quantile
def label (x):
    if x>q95:
        return 'VIP'
    elif (x>q75) & (x<q95):
        return "Preferred"
    else:
        np.nan

customer_amount['label']=customer_amount['amount_spent'].apply(label)
#customer_amount['label']=customer_amount['amount_spent'].apply(lambda x : 'VIP' if (x > q95) else np.nan)
customer_amount.head(2)

,amount_spent,Country,label
CustomerID,,,
12346,77183.6,United Kingdom,VIP
12347,4310.0,Iceland,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [122]:
# your code here
# First create a condition to filter de data by VIP customers. 
vip_cond=customer_amount['label']=='VIP'
# Then group by country and count the number of vips
customer_amount[vip_cond].groupby('Country')['label'].count().sort_values(ascending=False)

# United Kingdom is the country with most VIP customers

Country
United Kingdom     177
Germany             10
France               9
Switzerland          3
Spain                2
Portugal             2
Japan                2
EIRE                 2
Sweden               1
Singapore            1
Norway               1
Netherlands          1
Finland              1
Denmark              1
Cyprus               1
Channel Islands      1
Belgium              1
Australia            1
Name: label, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [129]:
# your code here
vip_preferred_cond=(customer_amount['label']=='VIP') | (customer_amount['label']=='Preferred')
customer_amount[vip_preferred_cond].groupby('Country')['label'].count().sort_values(ascending=False)

# United Kingdom is the country with most VIP+preferred customers

Country
United Kingdom     932
Germany             39
France              29
Belgium             11
Spain                9
Switzerland          9
Portugal             7
Norway               7
Finland              5
Italy                5
Channel Islands      4
Japan                4
Denmark              3
EIRE                 3
Australia            3
Cyprus               2
Sweden               2
Israel               2
Greece               1
Iceland              1
Lebanon              1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Canada               1
Austria              1
Name: label, dtype: int64